In [1]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
import sys
sys.path.insert(1, '../')

from utils.get_locations import HEALTH_FACILITY_TOWNS

In [ ]:
data_dir = '/Users/abdulkadirgokce/Documents/MSFeCare/MSF-eCARE-filedrop'

!ls $data_dir

In [3]:
filename = 'full-310k - Copy.csv'
file_dir = os.path.join(data_dir, filename)
df = pd.read_csv(file_dir)
df

/Users/abdulkadirgokce/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,2,5,8,9,43,46,48,51,54,114,129,130,131,134,135,136,154,156,158,160,182,183,184,186,187,191,202,203,204,208,212,234,248,252,265,266) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,source,project,hf,created,slot_id,a_age,a_gender2,a_weight,abdo_ds,abdo_inguin_mass,...,uti_vomiting,uvul_complication,uvulectomy,vacci_card,vaccines,vsd_added_signs,wfa,wfa2,wfa_less_than_neg3sd,wfh2
0,0,RCA-1,2,15feb2017 12:11:47,156.0,18.0,male,10.2,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,RCA-1,3,01mar2017 08:59:36,169.0,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,RCA-1,1,05jan2017 12:39:43,150.0,12.0,male,7.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,RCA-1,3,16aug2017 10:05:44,186.0,12.0,male,8.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,RCA-1,1,28apr2017 19:19:26,167.0,12.0,male,9.8,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310299,8,RCA-2,801,12mar2020 09:55:48,158.0,37,female,12.6,NaN,0,...,0.0,0.0,0.0,0.0,notime,NaN,NaN,0.0,0.0,NaN
310300,8,RCA-2,801,08jan2020 08:26:21,158.0,5,male,7.2,NaN,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN
310301,8,RCA-2,801,04apr2020 10:06:07,158.0,26,female,12.1,[1fafe800-fd5b-11e3-a3ac-0800200c9a66],0,...,0.0,0.0,0.0,0.0,notime,NaN,NaN,0.0,0.0,NaN
310302,8,RCA-2,801,16jul2020 14:42:21,158.0,17,male,10.0,NaN,0,...,0.0,0.0,0.0,1.0,incomplete,NaN,NaN,0.0,0.0,NaN


In [4]:
df.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310304 entries, 0 to 310303
Data columns (total 271 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    source                            object 
 1    project                           object 
 2    hf                                object 
 3    created                           object 
 4    slot_id                           float64
 5    a_age                             object 
 6    a_gender2                         object 
 7    a_weight                          float64
 8    abdo_ds                           object 
 9    abdo_inguin_mass                  object 
 10   abdo_intense_testispain           float64
 11   abdo_painkiller_response          float64
 12   adre_response                     float64
 13   agree_tx                          object 
 14   allergy_peni                      float64
 15   anaphyl_confirmed                 float64
 16   bloody_stool      

In [5]:
# Drop entries where too many features are not present

df = df.drop(df.index[df['source'].isin(['abdominal', 'eau', 'chaud', 'sro', 'coartem'])]) # 5 entries
df = df.drop(df.index[df['t_ab_recommended'].isna()]) # 8 entries
df = df.drop(df.index[df['a_gender2'].isna()]) # 10 entries
df = df.reset_index()

In [6]:
diagnoses = [ 
    'hisdx_severe',
    'hisdx_urti',
    'hisdx_lrti',
    'hisdx_cough_persist',
    'hisdx_diarrhwatery',
    'hisdx_diarrhblood',
    'hisdx_otitis',
    'hisdx_malaria_simple',
    'hisdx_malaria_sev',
    'hisdx_malnut',
    'hisdx_measles',
    'hisdx_anemia',
    'hisdx_others',
    'hisdx_ot_skin',
    'hisdx_ot_uti',
    'hisdx_ot_eye',
    'hisdx_ot_mouth',
    'hisdx_ot_fever_viral'
]

for diag in diagnoses:
    #print(df[diag].value_counts(dropna=False))
    pass

## Categorical features

In [7]:
neuro_features = [
    'd_unconscious',
    'd_convuls_his',
    'd_convuls_now',
    'danger_sign_neck',
    'danger_sign_interaction',
    'danger_sign_tonus',
    's_limp'
]


In [8]:
resp_features = [
    #'cough_ds',
    'cough_indrawing',
    'cough_malaise_his',
    'cough_prolonged',
    'cough_wheez_rhinitis',
    'cough_wheeze_rash',
    'cough_wheezing',
    #'cough_oxygen',
    #'cough_rr',
    'high_respiratory_rate',
    's_cough',
    'danger_sign_grunting',
]

In [9]:
# Replace bad entries with None
df.loc[df.index[df['hydration_thirst']=='574143ff-2257-438e-932f-291250d6c2cf'], 'hydration_thirst'] = np.nan
df.loc[df.index[df['hydration_thirst']=='9f1e50be-ea44-45bf-baba-15e6079b2268'], 'hydration_thirst'] = np.nan
df.loc[df.index[df['hydration_thirst']=='5ad025d5-b7fe-4fe6-ac2d-aa4c75914cb4'], 'hydration_thirst'] = np.nan
df.loc[df.index[df['abdo_ds']=='[1fafe800-fd5b-11e3-a3ac-0800200c9a66]'], 'abdo_ds'] = np.nan

df.loc[df.index[df['hydration_neuro']=='normal'], 'hydration_neuro'] = 0
df.loc[df.index[df['hydration_skin']=='normal'], 'hydration_skin'] = 0
df.loc[df.index[df['hydration_thirst']=='normal'], 'hydration_thirst'] = 0

abdo_features = [
    'hydration_assessed',
    'hydration_eye',
    'hydration_neuro',
    'hydration_skin',
    'hydration_thirst',
    'diarrhoea_prolonged',
    'bloody_stool',
    #'abdo_ds',
    's_diarr',
    's_abdopain',
    's_vomit',
    'danger_sign_vomit_confirmed',
    'danger_sign_vomit_only',
    'danger_sign_vomiting'
]


In [10]:
df.loc[df.index[df['ear_pus']=='earpus_none'], 'ear_pus'] = 0
ear_features = [
    'ear_pain_new',
    'ear_pain_prolonged',
    'ear_tender_swelling',
    'ear_pus',
    's_earpain'   
]

In [11]:
"""
group as muac_low
--s_muac_orange
--s_muac_red
"""

muac_low = ((df['s_muac']=='orange') | (df['s_muac']=='red')).astype(int).rename('muac_low')

In [12]:
"""
redo as hb_low if lab_hb is not <9
--lab_hb
"""
hb_low = (df['lab_hb']<9).astype(int).rename('hb_low')

In [13]:
df.loc[df.index[df['eye_sympt']=='[1fafe800-fd5b-11e3-a3ac-0800200c9a66]'], 'eye_sympt'] = np.nan
eye_features = ['eye_sympt', 's_eyepb']

In [14]:
nonspecific_features = ['s_none', 'no_symptom_classifying_reported', 'no_symptom_ecare_reported']
measles_features = ['ms_measles']
skin_features = ['s_skin']
urine_features = ['lab_urine_pos', 's_dysuria', 's_hematuria']

In [15]:
categorical_features = [
    'a_gender2',
    'anaphyl_confirmed',
    'danger_sign_jaundice',
    'fever_prolonged',
    'lab_malaria_pos',
    'mouth_trush',
    's_oedema',
    's_pallor',
    's_drepano',
    # 's_fever_any' - Not present in dataset,
    's_throat',
    's_mouthpb',
    's_joint',
    'sam_u6add_ocp',
    'wfa_less_than_neg3sd'
]

In [16]:
categorical_features += neuro_features + measles_features + nonspecific_features \
                     +  resp_features + abdo_features + ear_features + eye_features \
                     +  skin_features + urine_features

In [19]:
categorical_data = pd.get_dummies(
    df[categorical_features], columns=categorical_features, dummy_na=True
).rename(
    lambda col: col.replace('.0', ''), axis=1
)
categorical_data

,a_gender2_female,a_gender2_male,a_gender2_nan,anaphyl_confirmed_0,anaphyl_confirmed_1,anaphyl_confirmed_nan,danger_sign_jaundice_0,danger_sign_jaundice_1,danger_sign_jaundice_nan,fever_prolonged_0,...,s_skin_nan,lab_urine_pos_0,lab_urine_pos_1,lab_urine_pos_nan,s_dysuria_0,s_dysuria_1,s_dysuria_nan,s_hematuria_0,s_hematuria_1,s_hematuria_nan
0,0,1,0,0,0,1,0,0,1,1,...,0,1,0,0,1,0,0,1,0,0
1,0,1,0,0,0,1,0,0,1,1,...,0,1,0,0,1,0,0,1,0,0
2,0,1,0,0,0,1,0,0,1,1,...,0,1,0,0,1,0,0,1,0,0
3,0,1,0,0,0,1,0,0,1,1,...,0,1,0,0,1,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,1,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310284,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
310285,0,1,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
310286,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
310287,0,1,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0


## Spatial features

In [ ]:
spatial_features = [
    'source',
    'project',
    'hf'
]


In [ ]:
file_path = 'gps_coordinates.json'
if os.path.exists(file_path):
    with open(file_path, 'r') as file:
        gps_coordinates = json.load(file)
else:
    %run get_locations.py
    with open(file_path, 'r') as file:
        gps_coordinates = json.load(file)

gps_coordinates

In [ ]:
df['hf'] = df['hf'].astype(int)

spatial_data = pd.DataFrame.from_dict(
    {
        'country': [project.split('-')[0] for project in df['project']],
        'hf_town': [HEALTH_FACILITY_TOWNS[hf] for hf in df['hf']],
        'latitude': [gps_coordinates[HEALTH_FACILITY_TOWNS[hf]][0] for hf in df['hf']],
        'longitude': [gps_coordinates[HEALTH_FACILITY_TOWNS[hf]][1] for hf in df['hf']]
    }
)

spatial_feats_for_pred = ['latitude', 'longitude']

spatial_data

## Temporal features

In [ ]:
temporal_features = [
    'created'
]

In [ ]:
datetimes = pd.to_datetime(df['created'], format='%d%b%Y %H:%M:%S')

temporal_data = pd.DataFrame()

#### Add $\Big(\sin\left(\frac{2\pi}{52}\textrm{week_number}\right),  \cos\left(\frac{2\pi}{52}\textrm{week_number}\right)\Big)$

In [ ]:
temporal_data['week_number_sine'] = [np.sin((2*np.pi/52)*date.week) for date in datetimes]
temporal_data['week_number_cosine'] = [np.cos((2*np.pi/52)*date.week) for date in datetimes]

#### Add $\Big(\sin\left(\frac{2\pi}{12}\textrm{month}\right),  \cos\left(\frac{2\pi}{12}\textrm{month}\right)\Big)$

In [ ]:
temporal_data['month_sine'] = [np.sin((2*np.pi/12)*date.month) for date in datetimes]
temporal_data['month_cosine'] = [np.cos((2*np.pi/12)*date.month) for date in datetimes]

In [ ]:
temporal_data['year'] = [date.year for date in datetimes]
temporal_data['year'] = temporal_data['year'].astype(float)
temporal_data['datetime'] = datetimes

In [ ]:
temporal_feats_for_pred = [
    'week_number_sine',
    'week_number_cosine',
    'month_sine',
    'month_cosine',
    'year'
]

In [ ]:
temporal_data

## Numerical features

In [ ]:
numerical_features = [
    'a_age',
    #'a_weight'
]

#### Median Imputation
Instead of mean use median imputation to fill missing data since median is robust to outliers

In [ ]:
df[numerical_features] = df[numerical_features].astype(float)
numerical_data = df[numerical_features].fillna(df[numerical_features].median())

#### Normalization

In [ ]:
# Save statistics of features before normalization
normalization_stats = {}
normalization_stats['numerical_data_mean'] = numerical_data.mean().to_dict()
normalization_stats['numerical_data_std'] = numerical_data.std().to_dict()

normalization_stats['spatial_data_mean'] = spatial_data[spatial_feats_for_pred].mean().to_dict()
normalization_stats['spatial_data_std'] = spatial_data[spatial_feats_for_pred].std().to_dict()

normalization_stats['temporal_data_mean'] = temporal_data[temporal_feats_for_pred].mean().to_dict()
normalization_stats['temporal_data_std'] = temporal_data[temporal_feats_for_pred].std().to_dict()

normalization_stats

In [ ]:
numerical_data = (numerical_data-numerical_data.mean())/numerical_data.std()

spatial_data[spatial_feats_for_pred] -= spatial_data[spatial_feats_for_pred].mean()
spatial_data[spatial_feats_for_pred] /= spatial_data[spatial_feats_for_pred].std()

temporal_data[temporal_feats_for_pred] -= temporal_data[temporal_feats_for_pred].mean()
temporal_data[temporal_feats_for_pred] /= temporal_data[temporal_feats_for_pred].std()

## Clean DataFrame

In [ ]:
spatial_data.info(True)

In [ ]:
temporal_data.info(True)

In [ ]:
numerical_data.info(True)

In [ ]:
categorical_data.info(True)

In [ ]:
patient_data = pd.concat(
    [
        spatial_data, 
        temporal_data, 
        numerical_data, 
        categorical_data.astype(np.float64),
        df[diagnoses]
    ], 
    axis=1
)

patient_data = patient_data

In [ ]:
patient_data.info(True)

In [ ]:
patient_data.to_csv(os.path.join(data_dir, 'patient_pred_data.csv'), index=False)

In [ ]:
metadata = {
    'spatial_features': list(spatial_data.columns),
    'spatial_feats_for_pred' : spatial_feats_for_pred,
    'temporal_features': list(temporal_data.columns),
    'temporal_feats_for_pred' : temporal_feats_for_pred,
    'numerical_features': list(numerical_data.columns),
    'categorical_features': list(categorical_data.columns),
    'diagnoses' : diagnoses,
    'normalization_stats' : normalization_stats
    
}

with open(os.path.join(data_dir, 'metadata_patient_pred.json'), 'w') as file:
    json.dump(metadata, file, indent=4)

In [ ]:
metadata